In [ ]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing

# Обработка DataFrame

**Задача 1**

На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad средствами библиотеки Pandas сформируйте поле выводов по каждому анализирумомому параметру. Например, по показателю pH получен результат 8.4 единицы pH при нормативе от 6 до 9 единиц pH. Т.о. по данному показателю результат анализа в норме. Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, 6 >= pH >= 9. В итоговом DataFrame столбец "Показатель" сделайте индексным.

In [ ]:
# загружаем данные
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=',', index_col ='Показатель' )
df

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,б/цвета,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [ ]:
# разобьем "норматив" по столцам
sep_columns = df['Норматив'].str.split(expand=True)
temporary_df = sep_columns[2].str.split('-', expand=True)

In [ ]:
# из полученных столбцов сделаем датафрейм условии
sep_columns['Условие'] = sep_columns[0] + ' ' + sep_columns[1]
sep_columns['Левая граница'] = temporary_df[0]
sep_columns['Правая граница'] = temporary_df[1]
condition_df = sep_columns.drop([0, 1, 2], axis=1)
condition_df

,Условие,Левая граница,Правая граница
Показатель,,,
pH,в пределах,6,9
Запах,не более,2,3
Цветность,не более,30,None
Жёсткость,в пределах,7,10
Аммиак и аммоний-ион (по азоту),не более,"1,5",None
Нитриты (по NO2),не более,"3,3",None
Нитраты (по NO3),не более,45,None
Фосфаты (P),не более,"3,5",None
Хлориды (Cl),не более,350,None


In [ ]:
# правую границу с пустыми значениями заполним значениями из левой границы
mask = condition_df['Правая граница'].isnull()
condition_df.loc[mask, 'Правая граница'] = condition_df.loc[mask, 'Левая граница']
condition_df

,Условие,Левая граница,Правая граница
Показатель,,,
pH,в пределах,6,9
Запах,не более,2,3
Цветность,не более,30,30
Жёсткость,в пределах,7,10
Аммиак и аммоний-ион (по азоту),не более,"1,5","1,5"
Нитриты (по NO2),не более,"3,3","3,3"
Нитраты (по NO3),не более,45,45
Фосфаты (P),не более,"3,5","3,5"
Хлориды (Cl),не более,350,350


In [ ]:
# объединим начальный df и df условий
df = pd.concat([df, condition_df], axis=1)
df

,Единица измерений,Результат анализа,Норматив,Условие,Левая граница,Правая граница
Показатель,,,,,,
pH,единицы pH,8.4,в пределах 6-9,в пределах,6,9
Запах,баллы,1,не более 2-3,не более,2,3
Цветность,градусы,б/цвета,не более 30,не более,30,30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,в пределах,7,10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",не более,"1,5","1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,"3,3","3,3"
Нитраты (по NO3),мг/дм3,24,не более 45,не более,45,45
Фосфаты (P),мг/дм3,0.36,"не более 3,5",не более,"3,5","3,5"
Хлориды (Cl),мг/дм3,200,не более 350,не более,350,350


In [ ]:
# заменим строковое значение в числовом столце на число
df.at['Цветность', 'Результат анализа'] = 0
df

,Единица измерений,Результат анализа,Норматив,Условие,Левая граница,Правая граница
Показатель,,,,,,
pH,единицы pH,8.4,в пределах 6-9,в пределах,6,9
Запах,баллы,1,не более 2-3,не более,2,3
Цветность,градусы,0,не более 30,не более,30,30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,в пределах,7,10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",не более,"1,5","1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,"3,3","3,3"
Нитраты (по NO3),мг/дм3,24,не более 45,не более,45,45
Фосфаты (P),мг/дм3,0.36,"не более 3,5",не более,"3,5","3,5"
Хлориды (Cl),мг/дм3,200,не более 350,не более,350,350


In [ ]:
df['Левая граница'] = df['Левая граница'].str.replace(',', '.')
df['Правая граница'] = df['Правая граница'].str.replace(',', '.')
df

,Единица измерений,Результат анализа,Норматив,Условие,Левая граница,Правая граница
Показатель,,,,,,
pH,единицы pH,8.4,в пределах 6-9,в пределах,6,9
Запах,баллы,1,не более 2-3,не более,2,3
Цветность,градусы,0,не более 30,не более,30,30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,в пределах,7,10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",не более,1.5,1.5
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,3.3,3.3
Нитраты (по NO3),мг/дм3,24,не более 45,не более,45,45
Фосфаты (P),мг/дм3,0.36,"не более 3,5",не более,3.5,3.5
Хлориды (Cl),мг/дм3,200,не более 350,не более,350,350


In [ ]:
df.dtypes

Единица измерений    object
Результат анализа    object
Норматив             object
Условие              object
Левая граница        object
Правая граница       object
dtype: object

In [ ]:
# скорректируем тип данных столбцов
df['Единица измерений'] = df['Единица измерений'].astype(str)
df['Результат анализа'] = df['Результат анализа'].astype(float)
df['Норматив'] = df['Норматив'].astype(str)
df['Условие'] = df['Условие'].astype(str)
df['Левая граница'] = df['Левая граница'].astype(float)
df['Правая граница'] = df['Правая граница'].astype(float)

In [ ]:
df.dtypes

Единица измерений     object
Результат анализа    float64
Норматив              object
Условие               object
Левая граница        float64
Правая граница       float64
dtype: object

In [ ]:
# используем условие для заполнения столбца качества
mask_positiv = (df['Условие'] == 'не более') & (df['Правая граница'] > df['Результат анализа'])
mask_negativ = (df['Условие'] == 'не более') & (df['Правая граница'] < df['Результат анализа'])
df.loc[mask_positiv, 'Качество'] = 'Хорошо'
df.loc[mask_negativ, 'Качество'] = 'Плохо'
df

,Единица измерений,Результат анализа,Норматив,Условие,Левая граница,Правая граница,Качество
Показатель,,,,,,,
pH,единицы pH,8.400,в пределах 6-9,в пределах,6.0,9.0,NaN
Запах,баллы,1.000,не более 2-3,не более,2.0,3.0,Хорошо
Цветность,градусы,0.000,не более 30,не более,30.0,30.0,Хорошо
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в пределах,7.0,10.0,NaN
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",не более,1.5,1.5,Хорошо
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,3.3,3.3,Хорошо
Нитраты (по NO3),мг/дм3,24.000,не более 45,не более,45.0,45.0,Хорошо
Фосфаты (P),мг/дм3,0.360,"не более 3,5",не более,3.5,3.5,Хорошо
Хлориды (Cl),мг/дм3,200.000,не более 350,не более,350.0,350.0,Хорошо


In [ ]:
# используем условие для заполнения столбца качества
mask_positiv = (
    (df['Условие'] == 'в пределах') & 
    (df['Правая граница'] >= df['Результат анализа']) & 
    (df['Левая граница'] <= df['Результат анализа'])
    )

df.loc[mask_positiv, 'Качество'] = 'Хорошо'
df

,Единица измерений,Результат анализа,Норматив,Условие,Левая граница,Правая граница,Качество
Показатель,,,,,,,
pH,единицы pH,8.400,в пределах 6-9,в пределах,6.0,9.0,Хорошо
Запах,баллы,1.000,не более 2-3,не более,2.0,3.0,Хорошо
Цветность,градусы,0.000,не более 30,не более,30.0,30.0,Хорошо
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в пределах,7.0,10.0,Хорошо
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",не более,1.5,1.5,Хорошо
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,3.3,3.3,Хорошо
Нитраты (по NO3),мг/дм3,24.000,не более 45,не более,45.0,45.0,Хорошо
Фосфаты (P),мг/дм3,0.360,"не более 3,5",не более,3.5,3.5,Хорошо
Хлориды (Cl),мг/дм3,200.000,не более 350,не более,350.0,350.0,Хорошо


In [ ]:
# удалим лишние столцы
df = df.drop(['Условие', 'Левая граница', 'Правая граница'], axis=1)
df

,Единица измерений,Результат анализа,Норматив,Качество
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,Хорошо
Запах,баллы,1.000,не более 2-3,Хорошо
Цветность,градусы,0.000,не более 30,Хорошо
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,Хорошо
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",Хорошо
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Хорошо
Нитраты (по NO3),мг/дм3,24.000,не более 45,Хорошо
Фосфаты (P),мг/дм3,0.360,"не более 3,5",Хорошо
Хлориды (Cl),мг/дм3,200.000,не более 350,Хорошо


# Теория вероятности. События

Задача 2

* В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?
* В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

In [ ]:
import random

In [ ]:
num_experiments = 1_000_000 
num_fruit = 3            
result_experiments = []
# проводим эксперименты
for experiment in range(1, num_experiments + 1):
  temporary_list = []
  # заполняем ящик
  box = ["apple"] * 4 + ['orange'] * 5 
  # вытаскиваем 3 фрукта из ящика
  for _ in range(num_fruit):
    fruit = random.choice(box)
    temporary_list.append(fruit)
    box.remove(fruit)
  result_experiments.append(temporary_list)
# формируем DataFrame с результатами
result_df = pd.DataFrame(result_experiments)
result_df.head()

,0,1,2
0,orange,orange,apple
1,orange,apple,orange
2,orange,apple,orange
3,orange,orange,orange
4,orange,orange,apple


In [ ]:
# формируем условие где 3 фрукта апельсины
mask = ((result_df[0] == 'orange') &
        (result_df[1] == 'orange') &
        (result_df[2] == 'orange'))

In [ ]:
# считаем вероятность
result_df.loc[mask, 'Result'] = 1
prob = result_df['Result'].sum() / num_experiments
print('Вероятность вытащить из ящика 3 апельсина подряд - {}'.format(prob))

Вероятность вытащить из ящика 3 апельсина подряд - 0.119311


Задача 3

* Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?
* В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
num_experiments = 100000
num_details = 2
result_experiments = []
# проводим эксперименты
for experiment in range(1, num_experiments + 1):
  temporary_list = []
  # создаем список деталей
  details = ['standard'] * 7 + ['non-standard'] * 3
  # проверяем детали
  for _ in range(num_details):
    detail = random.choice(details)
    temporary_list.append(detail)
    details.remove(detail)
  result_experiments.append(temporary_list)
# формируем DataFrame с результатами
result_df = pd.DataFrame(result_experiments)
result_df.head()


,0,1
0,standard,standard
1,standard,non-standard
2,standard,standard
3,non-standard,standard
4,standard,standard


In [ ]:
# считаем вероятность
prob = len(result_df[(result_df[0] == 'non-standard') &
          (result_df[1] == 'standard')]) / num_experiments
print('Вероятность что вторая деталь стандартная - {}'.format(prob))

Вероятность что вторая деталь стандартная - 0.232
